In [102]:
import pandas as pd
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install transformers

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [95]:
df = pd.read_csv('news_cleaned.csv', sep = ";")

In [96]:
df = df.drop(columns=['Unnamed: 0'])

In [97]:
def split_into_sentences(s):
    # Remove the leading and trailing square brackets and single quotes
    s = s[2:-2]

    # Split the string into a list of sentences
    sentences = s.split("', '")

    return sentences

# Apply the function to the 'sentences' column
df['sentences'] = df['sentences'].apply(split_into_sentences)

In [118]:
AAPL_df = df[df['ticker'] == 'AAPL']
samples_100_AAPL = AAPL_df.sample(100)

In [110]:
import torch

In [119]:
def classify_sentences(sentences):
    # Initialize the lists of headlines and scores
    predictions = []
    positives = []
    negatives = []
    neutrals = []

    # Process each sentence
    for sentence in sentences:
        # Tokenize the sentence
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

        # Get the model's prediction
        outputs = model(**inputs)

        # Get the predicted scores
        scores = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = outputs.logits.argmax(dim=-1).item()

        # Add the sentence and scores to the lists
        predictions.append(prediction)
        positives.append(scores[:, 0].item())
        negatives.append(scores[:, 1].item())
        neutrals.append(scores[:, 2].item())

    return predictions, positives, negatives, neutrals

# Apply the function to the 'sentences' column and concatenate the results
samples_100_AAPL[['Predictions', 'Positive', 'Negative', 'Neutral']] = samples_100_AAPL['sentences'].apply(lambda x: pd.Series(classify_sentences(x)))

In [117]:
samples_100_AAPL

,id,company,title,published,ticker,company_words,sentences,Predictions,Positive,Negative,Neutral
14110,4c7f9780ebe819f58ac6765b8efabfa2666c75fbb7e275...,Apple_Inc,Netflix Falls to Earth; Note to the Biden Fede...,2022-04-20 22:19:00.000,AAPL,apple,[but new streaming platformsnbcs peacock param...,[0],[0.6743584871292114],[0.0105115482583642],[0.31512996554374695]
7046,39ab66fad944281724ad8c71a810e67ebbde3dd7d855ad...,Apple_Inc,The ten trillion dollar man: how Larry Fink be...,2021-10-07 04:00:41.953,AAPL,apple,[ today blackrocks profit margins are fatter t...,[1],[0.07949826121330261],[0.8025999069213867],[0.11790191382169724]
21502,31ed829c92730d0674a4ce0ab9358cced42c0256d22842...,Apple_Inc,Even JPMorgan's bullish strategist Kolanovic s...,2022-11-15 14:15:00.000,AAPL,apple,[ ticker security name tsla tesla amc ...,[2],[0.10110578685998917],[0.014380681328475475],[0.8845135569572449]
22357,ba5373f46d2916f362bad06ba7570c5ad7e269a5c9ad82...,Apple_Inc,Shanghai Disneyland Reopens as Controls Ease ...,2022-12-09 07:03:00.000,AAPL,apple,"[and apple inc, apple has begun acceleratin...","[2, 0]","[0.03302522003650665, 0.9020821452140808]","[0.024208195507526398, 0.02150355465710163]","[0.9427666068077087, 0.07641434669494629]"
18386,dc4792bdece783adf60032c3e4e84fe02f6a46ca7c1a27...,Apple_Inc,Biden Describes Trump's Governing Philosophy a...,2022-08-26 00:06:00.000,AAPL,apple,[],[2],[0.35916322469711304],[0.21665172278881073],[0.4241850972175598]
9202,d571b84bebce7931ce315719759b90de52806286cba6b0...,Apple_Inc,FTC Sues To Block Merger Of Chip Firms --...,2021-12-03 07:14:00.000,AAPL,apple,"[companies such as apple inc, as the u.s.s big...","[2, 2]","[0.06070193648338318, 0.16213782131671906]","[0.023333294317126274, 0.10379832237958908]","[0.9159647822380066, 0.7340638637542725]"
19783,3c6f3307cdcbf24536da7abd051b64d67d496ee0c477ba...,Apple_Inc,Rich millennials say this is the best long-ter...,2022-10-11 13:01:00.000,AAPL,apple,[],[2],[0.35916322469711304],[0.21665172278881073],[0.4241850972175598]
5332,313334478c643cd0d78315b6d9bc031e59fe43332360d8...,Apple_Inc,The real reason most super apps are not super ...,2021-08-24 04:00:52.422,AAPL,apple,[there are more than two million apps in apple...,"[2, 1, 2]","[0.06786294281482697, 0.015922728925943375, 0....","[0.015705404803156853, 0.9346423745155334, 0.0...","[0.9164316058158875, 0.049434807151556015, 0.7..."
4030,2b3f810511a5b8ec780ce9038417d94f200f1dac332984...,Apple_Inc,Voters Can Spot Radical Left-Wing Policy,2021-05-06 00:00:00.000,AAPL,apple,[bidens west wing ideologues of the notion tha...,[1],[0.04369858279824257],[0.6033419966697693],[0.35295945405960083]
6070,f334fda6a1275478e19f198df8d0f1a513bfa7c9e78560...,Apple_Inc,5G Deals Charge Up Sales for iPhones,2021-09-13 00:00:00.000,AAPL,apple,[the newest iphones set to be showcased at ap...,"[2, 0, 0, 0, 2]","[0.3014761209487915, 0.9041233062744141, 0.831...","[0.010132420808076859, 0.011704755015671253, 0...","[0.6883915066719055, 0.0841718465089798, 0.106..."


In [120]:
samples_100_AAPL.to_csv('output.csv', index=True)